In [0]:
import os, time, gc, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
from sklearn.utils import resample

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score


import xgboost as xgb
import matplotlib.pyplot as plt

pd.set_option('max_rows', 9999)
pd.set_option('max_columns', 9999)

start = time.time()

In [93]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [94]:
cd /content/gdrive/My Drive/ML 4

/content/gdrive/My Drive/ML 4


In [0]:
train = pd.read_csv("transaction_id_combined.csv")

In [96]:
cd /content/gdrive/My Drive/ML 4/Test

/content/gdrive/My Drive/ML 4/Test


In [97]:
ls

 cdmb_hyunji_Version_underlgbm_0_89_edited.ipynb
 sample_submission.csv
 submission.csv
'test_combined (1).csv'
 test_combined.csv
'test_combined(Yaehee_version).csv'
'Test_Please (1).csv'
 test_region.csv
'train_combined(Yaehee_version).csv'


In [ ]:
test = pd.read_csv("Test_Please.csv")

In [0]:
not_fraud = train[train.isFraud==0]
fraud = train[train.isFraud==1]

In [0]:
# Config
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
params = {
    'task': 'train',
    'objective': 'binary',
    'metrics': 'auc',
    'max_depth': 9,
    'learning_rate': 0.15,
    'random_state': 0,
    'bagging_fraction': 0.9,
    'feature_fraction': 0.9,
    'min_data_in_leaf': 36,
    'num_Leaves': 60,
}
config = {
    'num_boost_round':  500,
    'early_stopping_rounds': 100,
    'verbose_eval': 2000
}

In [0]:
def underlgbm():
  not_fraud_downsampled = resample(not_fraud,
                               n_samples = len(fraud)) # match minority n
  downsampled = pd.concat([not_fraud_downsampled, fraud])
  # make Dataset
  features = [c for c in downsampled.columns if c not in ['TransactionID', 'isFraud']]
  target = 'isFraud'

  X = downsampled[features].values
  Y = downsampled[target].values

  X_test = test[features].values

  oof = np.zeros(len(downsampled))
  preds = np.zeros(len(test))
  feature_importance = np.zeros(len(features))      

  
  

  
    # %%time
  # Model training
  for i, (trn_index, val_index) in enumerate(cv.split(X, Y)):
      print('{} Folds'.format(i + 1))

      _start = time.time()
      X_train, Y_train = X[trn_index], Y[trn_index]
      X_valid, Y_valid = X[val_index], Y[val_index]

      trn_data = lgb.Dataset(X_train, label=Y_train)
      val_data = lgb.Dataset(X_valid, label=Y_valid, reference=trn_data)

      model = lgb.train(params, trn_data, valid_sets=[val_data, trn_data], valid_names=['eval', 'train'], **config)

      oof[val_index] = model.predict(X_valid)
      preds += model.predict(X_test, iteration=model.best_iteration) / cv.get_n_splits()
      feature_importance += model.feature_importance(iteration=model.best_iteration)

      elapsedtime = time.time() - _start
      s = datetime.timedelta(seconds=elapsedtime)

      print('{} Folds Running Time: {}'.format(i + 1, str(s)))
      print('#' * 50)

      del model
      gc.collect()
      
  print('Accuracy:{}'.format(accuracy_score(downsampled['isFraud'], oof > 0.5)),
    'Precision:{}'.format(precision_score(downsampled['isFraud'], oof > 0.5)),
    'Recall:{}'.format(recall_score(downsampled['isFraud'], oof > 0.5)),
    'F1:{}'.format(f1_score(downsampled['isFraud'], oof > 0.5)),
   'ROC_AUC:{}'.format(roc_auc_score(downsampled['isFraud'], oof > 0.5)), sep = '\n')

  return preds



In [90]:
prediction = underlgbm()

1 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990173	eval's auc: 0.939505
1 Folds Running Time: 0:00:14.735835
##################################################
2 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.99101	eval's auc: 0.930377
2 Folds Running Time: 0:00:14.987530
##################################################
3 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990609	eval's auc: 0.933036
3 Folds Running Time: 0:00:14.677192
##################################################
4 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.991064	eval's auc: 0.931673
4 Folds Running Time: 0:00:14.892083
############################

In [91]:
#average

num_iteration = 5
prediction = np.zeros(len(test))

for i in range(num_iteration):
  pred_temp = underlgbm()
  prediction += pred_temp

prediction_avg = prediction / num_iteration

sub = pd.read_csv('sample_submission.csv')
sub['isFraud'] = prediction_avg
sub.to_csv('submission_average.csv', index=False)
  

1 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990763	eval's auc: 0.937559
1 Folds Running Time: 0:00:15.271195
##################################################
2 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990959	eval's auc: 0.931149
2 Folds Running Time: 0:00:14.825570
##################################################
3 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990446	eval's auc: 0.934872
3 Folds Running Time: 0:00:14.937296
##################################################
4 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.991071	eval's auc: 0.93154
4 Folds Running Time: 0:00:14.757128
############################

In [99]:
# #max value

# num_iteration = 5
# prediction = np.zeros(len(test))

# for i in range(num_iteration):
#   pred_temp = underlgbm()
#   prediction = np.maximum(prediction, pred_temp)
  
# sub = pd.read_csv('sample_submission.csv')
# sub['isFraud'] = prediction
# sub.to_csv('submission_max.csv', index=False)


1 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990411	eval's auc: 0.932061
1 Folds Running Time: 0:00:15.522064
##################################################
2 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990738	eval's auc: 0.930346
2 Folds Running Time: 0:00:15.530665
##################################################
3 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990241	eval's auc: 0.934526
3 Folds Running Time: 0:00:15.553314
##################################################
4 Folds
Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[500]	train's auc: 0.990882	eval's auc: 0.928779
4 Folds Running Time: 0:00:15.539646
###########################